In [19]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

# 파일 불러와서 합치기 

In [ ]:

#인베스팅닷컴뉴스불러와서 합치기 ! 

In [13]:
pwd

'C:\\Users\\bitcamp\\Project4\\카테고리분류모델피클'

In [ ]:
cd C:\Users\bitcamp\Project4\data\investing.com news-20220425T032341Z-001\investing.com news\commodities-news
    

In [20]:
import os
data_lis=os.listdir('./')
data_lis

['news_commodities-news_00001.csv',
 'news_commodities-news_00002.csv',
 'news_commodities-news_00003.csv',
 'news_commodities-news_00004.csv',
 'news_commodities-news_00005.csv',
 'news_commodities-news_00006.csv',
 'news_commodities-news_00007.csv',
 'news_commodities-news_00008.csv',
 'news_commodities-news_00009.csv',
 'news_commodities-news_00010.csv',
 'news_commodities-news_00011.csv',
 'news_commodities-news_00012.csv',
 'news_commodities-news_00013.csv',
 'news_commodities-news_00014.csv',
 'news_commodities-news_00015.csv',
 'news_commodities-news_00016.csv',
 'news_commodities-news_00017.csv',
 'news_commodities-news_00018.csv',
 'news_commodities-news_00019.csv',
 'news_commodities-news_00020.csv',
 'news_commodities-news_00021.csv',
 'news_commodities-news_00022.csv',
 'news_commodities-news_00023.csv']

In [21]:
# 반복문 이용해서 데이터 합치기 
# csv파일 여러개 읽어와서 합치기 
df_cnbc = pd.DataFrame()

for files in data_lis:
    # advertisement 빼고 합치기 
    df = pd.read_csv(files,index_col=0)
    df_cnbc = pd.concat([df_cnbc,df])    

In [35]:
df = df_cnbc
df = df.dropna().reset_index()
df_comm = df.sort_values('date')

# 토큰화하기 

In [36]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re

In [37]:
def preprocessing(docs):
    tokenizer = RegexpTokenizer('[\w]+')
    stop_words = stopwords.words('english')
    p_stemmer = PorterStemmer()  # 어근 복원 : runs, running, ran => run,  cars => car
#     p = re.compile("[0-9]+")
#     숫자제거해야됨
#     docs = [p.sub(i) for i in docs]
    docs = BeautifulSoup(docs, "html5lib").get_text()
    low = docs.lower()
    tokens = tokenizer.tokenize(low)
    stopped_tokens = [i for i in tokens if not i in stop_words]
    stopped_tokens = [re.sub('[^a-zA-Z0-9]',' ',i) for i in stopped_tokens]
    stemmer_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    one_rmv = [w for w in stemmer_tokens if len(w)>1]
  
    return one_rmv

In [40]:
# 함수적용
df['tokened']  = df['content'].apply(preprocessing)

In [41]:
df

,index,company,date,title,content,tokened
0,0,By Reuters,"<span>3 hours ago (Apr 17, 2022 05:31AM ET)</s...",Tunisia says countries offer help to tackle im...,By Tarek Amara\nTUNIS (Reuters) - Some countri...,"[tarek, amara, tuni, reuter, countri, offer, h..."
1,1,By Investing.com,"<span>5 hours ago (Apr 17, 2022 03:32AM ET)</s...",Energy & Precious Metals - Weekly Review and O...,By Barani Krishnan\nInvesting.com -- Will they...,"[barani, krishnan, invest, com, europ, forthco..."
2,2,By Reuters,"<span>8 hours ago (Apr 17, 2022 12:41AM ET)</s...",Mexico's Pemex says fire under control at Sali...,MEXICO CITY (Reuters) -Mexican state oil compa...,"[mexico, citi, reuter, mexican, state, oil, co..."
3,3,By Reuters,"<span>14 hours ago (Apr 16, 2022 06:05PM ET)</...",New bird flu cases identified in Pennsylvania ...,By Kanishka Singh\nWASHINGTON (Reuters) - New ...,"[kanishka, singh, washington, reuter, new, bir..."
4,5,By Reuters,"<span>20 hours ago (Apr 16, 2022 12:15PM ET)</...","Eyeing green vote, Macron vows to exit oil, co...","By Michel Rose\nMARSEILLE, France (Reuters) - ...","[michel, rose, marseil, franc, reuter, presid,..."
...,...,...,...,...,...,...
30818,1018,By Reuters,"<span>Nov 22, 2010 03:04PM ET</span>","METALS-Copper slips on dollar, China demand wo...",* China demand for refined copper down in Oct...,"[china, demand, refin, copper, octob, sign, po..."
30819,1019,By Reuters,"<span>Nov 15, 2010 12:44AM ET</span>",GRAINS-Corn up 2 pct on talk of Chinese buying...,* Corn jumps on talk of China's purchases of ...,"[corn, jump, talk, china, purchas, latam, carg..."
30820,1020,By Reuters,"<span>Nov 09, 2010 11:16AM ET</span>",PRECIOUS-Gold hits new record as euro zone fea...,"* Euro zone debt back in focus, eyes on G20\n...","[euro, zone, debt, back, focu, eye, g20, trade..."
30821,1021,By Reuters,"<span>Nov 04, 2010 03:48AM ET</span>","GRAINS-Soy rises 1 pct, wheat firms after Fed ...","* Soy up 1 pct, wheat rises 0.8 pct on Fed mo...","[soy, pct, wheat, rise, pct, fed, move, grain,..."


# 전처리하기 

In [ ]:
# 단어를 시퀀스 형태로 변환해주기 
x = tokenizer.texts_to_sequences(preprocessed_sentences)

In [ ]:
# 기사최대길이로 패딩해주기 
max_len = max([len(l) for l in df['토큰화한기사']])

x = pad_sequences(x, maxlen=max_len)

# 모델 실행

In [ ]:
import tensorflow as tf
import numpy as np

##########모델 로드

model = tf.keras.models.load_model('./best_mode5l.h5')

In [ ]:
y_predict = model.predict(x)

In [ ]:
y_predict

In [ ]:
# 예측라벨값 리스트에 담아줌
lis = []
for i in range(len(y_predict)):
    리스.append(np.argmax(y_predict[i]))

In [ ]:
# 예측한 라벨값중 가장 높은 빈도 구하기 

import collections
# apple3개, banana2개, cherry는 4개
# list = ['apple', 'apple', 'apple', 'banana', 'banana',
# 'cherry', 'cherry', 'cherry', 'cherry']
# 빈도수를 세려면?
counts = collections.Counter(리스)
print(counts)
# 가장 흔하게 등장하는 단어의 첫번째 값을 most_common(1)로 추출
# most_common(1) => [('cherry',4)] _리스트
# most_common(1)[0] => ('cherry', 4) _ 튜플
# most_common(1)[0][0] => cherry _ 튜플에서 첫번째 추출
print('가장 많이 예측한 라벨은?: ', counts.most_common(1)[0][0])


In [ ]:
# 컨텐트 하나하나 확인해보기 
print(collections.Counter(df['content'][0])) #ship

In [ ]:
# predict 라벨값 데이터프레임에 붙이기 
df['category'] = 리스

In [ ]:
# 데이터받아서 카테고리 지정 

In [5]:
topics = [[0],[26],[6,9,10,23,25,27],[16,30],[2,4,5,7,8,11,14,18]] # oil, ship, sc, steel, vh
def classifi(label):
    for key, topic in enumerate(topics):
        if label in topic:
            return key+100
    return label

In [8]:
label = set()
for i in range(33):
    label.add(classifi(i))
a = list(label)

In [ ]:
tmp = [0 for _ in range(len(a))]

In [10]:
tmp(b=1)

In [12]:
def labeling(y):
    a.index(y)
    tmp = [0 for _ in range(len(a))]    
    tmp(a.index(y) = 1)    
    return tmp 

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (Temp/ipykernel_7516/2858377248.py, line 4)

In [11]:
df['label'].apply(labeling)
#출력해보면 원핫벡터가나올것임 

In [12]:
pwd

'C:\\Users\\bitcamp\\Project4\\카테고리분류모델피클'

In [13]:
cd ../

C:\Users\bitcamp\Project4
